# Day 8: Linear Regression

## Week 2: Classical Machine Learning - Introduction

### Learning Objectives
By the end of this lesson, you will be able to:
1. Understand the mathematical foundations of linear regression
2. Implement linear regression from scratch using NumPy
3. Use Scikit-learn's LinearRegression model
4. Evaluate models using MSE, RMSE, MAE, and R² score
5. Build a house price prediction model

---

## Table of Contents

1. [Introduction to Linear Regression](#1.-Introduction-to-Linear-Regression)
2. [The Mathematics Behind Linear Regression](#2.-The-Mathematics-Behind-Linear-Regression)
3. [Cost Function and Gradient Descent](#3.-Cost-Function-and-Gradient-Descent)
4. [Implementation from Scratch](#4.-Implementation-from-Scratch)
5. [Scikit-learn Implementation](#5.-Scikit-learn-Implementation)
6. [Model Evaluation Metrics](#6.-Model-Evaluation-Metrics)
7. [House Price Prediction Assignment](#7.-House-Price-Prediction-Assignment)
8. [Comparison and Analysis](#8.-Comparison-and-Analysis)

---

## Setup

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn imports
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Display settings
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")

---
## 1. Introduction to Linear Regression

### What is Linear Regression?

Linear Regression is a **supervised learning** algorithm used for predicting a **continuous** target variable based on one or more input features. It assumes a **linear relationship** between the input variables (X) and the output variable (y).

### Types of Linear Regression

1. **Simple Linear Regression**: One independent variable
   - $y = mx + b$

2. **Multiple Linear Regression**: Multiple independent variables
   - $y = w_0 + w_1x_1 + w_2x_2 + ... + w_nx_n$

### Use Cases

- House price prediction
- Sales forecasting
- Stock price prediction
- Medical outcome prediction
- Customer lifetime value estimation

In [ ]:
# Create simple example data to visualize linear regression
np.random.seed(42)
X_simple = np.linspace(0, 10, 50)
y_simple = 2 * X_simple + 1 + np.random.normal(0, 1.5, 50)  # y = 2x + 1 + noise

# Visualize
plt.figure(figsize=(10, 6))
plt.scatter(X_simple, y_simple, color='blue', alpha=0.6, label='Data points')
plt.plot(X_simple, 2 * X_simple + 1, color='red', linewidth=2, label='True relationship: y = 2x + 1')
plt.xlabel('X (Feature)', fontsize=12)
plt.ylabel('y (Target)', fontsize=12)
plt.title('Simple Linear Regression Concept', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("The goal of linear regression is to find the best-fit line through the data points.")

---
## 2. The Mathematics Behind Linear Regression

### Simple Linear Regression: y = mx + b

The equation of a line:

$$\hat{y} = mx + b$$

Where:
- $\hat{y}$ = predicted value
- $m$ = slope (coefficient) - how much y changes for each unit change in x
- $x$ = input feature
- $b$ = y-intercept (bias) - value of y when x = 0

### Multiple Linear Regression

$$\hat{y} = w_0 + w_1x_1 + w_2x_2 + ... + w_nx_n$$

Or in vector notation:

$$\hat{y} = \mathbf{X} \cdot \mathbf{w} + b$$

Where:
- $\mathbf{X}$ = feature matrix (n_samples x n_features)
- $\mathbf{w}$ = weight vector (coefficients)
- $b$ = bias term

In [ ]:
# Visualize the effect of slope and intercept
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

x_range = np.linspace(0, 5, 100)

# Effect of changing slope (m)
ax1 = axes[0]
for m in [0.5, 1, 2, 3]:
    ax1.plot(x_range, m * x_range + 1, label=f'm = {m}')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_title('Effect of Slope (m) on Line\n(b = 1 fixed)', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 5)
ax1.set_ylim(0, 15)

# Effect of changing intercept (b)
ax2 = axes[1]
for b in [-1, 0, 1, 2, 3]:
    ax2.plot(x_range, 2 * x_range + b, label=f'b = {b}')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_title('Effect of Intercept (b) on Line\n(m = 2 fixed)', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 5)
ax2.set_ylim(-2, 15)

plt.tight_layout()
plt.show()

### Analytical Solution: Ordinary Least Squares (OLS)

For simple linear regression, the optimal parameters can be calculated directly:

**Slope:**
$$m = \frac{\sum_{i=1}^{n}(x_i - \bar{x})(y_i - \bar{y})}{\sum_{i=1}^{n}(x_i - \bar{x})^2} = \frac{Cov(x, y)}{Var(x)}$$

**Intercept:**
$$b = \bar{y} - m\bar{x}$$

For multiple linear regression, using matrix notation:

$$\mathbf{w} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}$$

This is called the **Normal Equation**.

In [ ]:
# Calculate parameters using OLS formulas
def calculate_ols_parameters(X, y):
    """
    Calculate slope and intercept using Ordinary Least Squares formulas
    
    Parameters:
    -----------
    X : array-like
        Input features (1D array for simple linear regression)
    y : array-like
        Target values
    
    Returns:
    --------
    m : float
        Slope
    b : float
        Intercept
    """
    # Calculate means
    x_mean = np.mean(X)
    y_mean = np.mean(y)
    
    # Calculate slope using formula: m = Cov(x,y) / Var(x)
    numerator = np.sum((X - x_mean) * (y - y_mean))
    denominator = np.sum((X - x_mean) ** 2)
    m = numerator / denominator
    
    # Calculate intercept: b = y_mean - m * x_mean
    b = y_mean - m * x_mean
    
    return m, b

# Calculate for our simple example
m_ols, b_ols = calculate_ols_parameters(X_simple, y_simple)

print("OLS Solution for Simple Linear Regression")
print("="*50)
print(f"True parameters: m = 2.0, b = 1.0")
print(f"Estimated parameters: m = {m_ols:.4f}, b = {b_ols:.4f}")
print(f"\nEquation: y = {m_ols:.4f}x + {b_ols:.4f}")

In [ ]:
# Visualize the OLS fit
plt.figure(figsize=(10, 6))
plt.scatter(X_simple, y_simple, color='blue', alpha=0.6, label='Data points')
plt.plot(X_simple, m_ols * X_simple + b_ols, color='red', linewidth=2, 
         label=f'OLS fit: y = {m_ols:.2f}x + {b_ols:.2f}')
plt.plot(X_simple, 2 * X_simple + 1, color='green', linewidth=2, linestyle='--', 
         label='True: y = 2x + 1')

# Show residuals for a few points
for i in range(0, len(X_simple), 10):
    y_pred = m_ols * X_simple[i] + b_ols
    plt.plot([X_simple[i], X_simple[i]], [y_simple[i], y_pred], 
             color='gray', linestyle=':', alpha=0.7)

plt.xlabel('X (Feature)', fontsize=12)
plt.ylabel('y (Target)', fontsize=12)
plt.title('OLS Linear Regression Fit', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---
## 3. Cost Function and Gradient Descent

### Cost Function (Loss Function)

The cost function measures how well our model fits the data. For linear regression, we use **Mean Squared Error (MSE)**:

$$J(m, b) = \frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y}_i)^2 = \frac{1}{n}\sum_{i=1}^{n}(y_i - (mx_i + b))^2$$

Our goal is to minimize this cost function.

### Why MSE?

1. **Differentiable**: Smooth surface for optimization
2. **Convex**: Has a single global minimum
3. **Penalizes larger errors more**: Squared term

### Gradient Descent

Gradient descent is an iterative optimization algorithm that finds the minimum of the cost function:

**Update Rules:**

$$m = m - \alpha \frac{\partial J}{\partial m}$$
$$b = b - \alpha \frac{\partial J}{\partial b}$$

Where $\alpha$ is the **learning rate**.

**Gradients:**

$$\frac{\partial J}{\partial m} = -\frac{2}{n}\sum_{i=1}^{n}x_i(y_i - \hat{y}_i)$$
$$\frac{\partial J}{\partial b} = -\frac{2}{n}\sum_{i=1}^{n}(y_i - \hat{y}_i)$$

In [ ]:
# Visualize the cost function surface
from mpl_toolkits.mplot3d import Axes3D

def compute_cost(X, y, m, b):
    """Compute MSE cost"""
    n = len(y)
    predictions = m * X + b
    cost = (1/n) * np.sum((y - predictions) ** 2)
    return cost

# Create grid of m and b values
m_range = np.linspace(0, 4, 50)
b_range = np.linspace(-2, 4, 50)
M, B = np.meshgrid(m_range, b_range)

# Compute cost for each combination
Z = np.zeros_like(M)
for i in range(len(m_range)):
    for j in range(len(b_range)):
        Z[j, i] = compute_cost(X_simple, y_simple, m_range[i], b_range[j])

# Plot cost surface
fig = plt.figure(figsize=(14, 5))

# 3D surface
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(M, B, Z, cmap='viridis', alpha=0.8)
ax1.scatter([m_ols], [b_ols], [compute_cost(X_simple, y_simple, m_ols, b_ols)], 
            color='red', s=100, label='Minimum')
ax1.set_xlabel('Slope (m)')
ax1.set_ylabel('Intercept (b)')
ax1.set_zlabel('Cost (MSE)')
ax1.set_title('Cost Function Surface', fontsize=12, fontweight='bold')

# Contour plot
ax2 = fig.add_subplot(122)
contour = ax2.contour(M, B, Z, levels=20, cmap='viridis')
ax2.clabel(contour, inline=True, fontsize=8)
ax2.scatter([m_ols], [b_ols], color='red', s=100, zorder=5, label=f'Minimum: ({m_ols:.2f}, {b_ols:.2f})')
ax2.set_xlabel('Slope (m)')
ax2.set_ylabel('Intercept (b)')
ax2.set_title('Cost Function Contour', fontsize=12, fontweight='bold')
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Implement gradient descent from scratch
def gradient_descent(X, y, learning_rate=0.01, n_iterations=1000, verbose=False):
    """
    Perform gradient descent to find optimal m and b
    
    Parameters:
    -----------
    X : array-like
        Input features
    y : array-like
        Target values
    learning_rate : float
        Step size for parameter updates
    n_iterations : int
        Number of iterations
    verbose : bool
        Print progress
    
    Returns:
    --------
    m, b : floats
        Optimized parameters
    history : dict
        Training history
    """
    n = len(y)
    
    # Initialize parameters randomly
    m = np.random.randn()
    b = np.random.randn()
    
    # Store history for visualization
    history = {'m': [m], 'b': [b], 'cost': []}
    
    for i in range(n_iterations):
        # Predictions
        y_pred = m * X + b
        
        # Compute cost
        cost = (1/n) * np.sum((y - y_pred) ** 2)
        history['cost'].append(cost)
        
        # Compute gradients
        dm = -(2/n) * np.sum(X * (y - y_pred))
        db = -(2/n) * np.sum(y - y_pred)
        
        # Update parameters
        m = m - learning_rate * dm
        b = b - learning_rate * db
        
        # Store history
        history['m'].append(m)
        history['b'].append(b)
        
        # Print progress
        if verbose and (i + 1) % (n_iterations // 10) == 0:
            print(f"Iteration {i+1}: Cost = {cost:.4f}, m = {m:.4f}, b = {b:.4f}")
    
    return m, b, history

# Run gradient descent
print("Running Gradient Descent...")
print("="*60)
m_gd, b_gd, history = gradient_descent(X_simple, y_simple, learning_rate=0.01, n_iterations=1000, verbose=True)

print(f"\nFinal Results:")
print(f"OLS Solution: m = {m_ols:.4f}, b = {b_ols:.4f}")
print(f"GD Solution:  m = {m_gd:.4f}, b = {b_gd:.4f}")

In [ ]:
# Visualize gradient descent convergence
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Cost over iterations
axes[0].plot(history['cost'], color='blue')
axes[0].set_xlabel('Iteration')
axes[0].set_ylabel('Cost (MSE)')
axes[0].set_title('Cost Function Convergence', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Parameter trajectory on contour
contour = axes[1].contour(M, B, Z, levels=20, cmap='viridis')
axes[1].plot(history['m'], history['b'], 'r.-', alpha=0.5, markersize=2)
axes[1].scatter(history['m'][0], history['b'][0], color='green', s=100, zorder=5, label='Start')
axes[1].scatter(history['m'][-1], history['b'][-1], color='red', s=100, zorder=5, label='End')
axes[1].set_xlabel('Slope (m)')
axes[1].set_ylabel('Intercept (b)')
axes[1].set_title('Gradient Descent Path', fontsize=12, fontweight='bold')
axes[1].legend()

# Parameters over iterations
axes[2].plot(history['m'], label='m (slope)', color='blue')
axes[2].plot(history['b'], label='b (intercept)', color='orange')
axes[2].axhline(y=m_ols, color='blue', linestyle='--', alpha=0.5, label=f'True m = {m_ols:.2f}')
axes[2].axhline(y=b_ols, color='orange', linestyle='--', alpha=0.5, label=f'True b = {b_ols:.2f}')
axes[2].set_xlabel('Iteration')
axes[2].set_ylabel('Parameter Value')
axes[2].set_title('Parameter Convergence', fontsize=12, fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Effect of Learning Rate

In [ ]:
# Compare different learning rates
learning_rates = [0.001, 0.01, 0.05, 0.1]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for i, lr in enumerate(learning_rates):
    _, _, hist = gradient_descent(X_simple, y_simple, learning_rate=lr, n_iterations=500)
    axes[i].plot(hist['cost'])
    axes[i].set_xlabel('Iteration')
    axes[i].set_ylabel('Cost (MSE)')
    axes[i].set_title(f'Learning Rate = {lr}', fontsize=11, fontweight='bold')
    axes[i].grid(True, alpha=0.3)
    
    # Check for divergence
    if hist['cost'][-1] > hist['cost'][0]:
        axes[i].text(0.5, 0.5, 'DIVERGING!', transform=axes[i].transAxes, 
                    fontsize=14, color='red', ha='center')

plt.suptitle('Effect of Learning Rate on Convergence', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---
## 4. Implementation from Scratch

Let's implement a complete Linear Regression class from scratch.

In [ ]:
class LinearRegressionScratch:
    """
    Linear Regression implemented from scratch using NumPy.
    
    Supports both OLS (analytical) and Gradient Descent optimization.
    """
    
    def __init__(self, learning_rate=0.01, n_iterations=1000, method='ols'):
        """
        Initialize Linear Regression model.
        
        Parameters:
        -----------
        learning_rate : float
            Learning rate for gradient descent
        n_iterations : int
            Number of iterations for gradient descent
        method : str
            'ols' for analytical solution, 'gd' for gradient descent
        """
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.method = method
        self.weights = None
        self.bias = None
        self.cost_history = []
        
    def fit(self, X, y):
        """
        Fit the linear regression model.
        
        Parameters:
        -----------
        X : array-like of shape (n_samples, n_features)
            Training data
        y : array-like of shape (n_samples,)
            Target values
        """
        # Ensure X is 2D
        X = np.array(X)
        y = np.array(y)
        
        if X.ndim == 1:
            X = X.reshape(-1, 1)
        
        n_samples, n_features = X.shape
        
        if self.method == 'ols':
            self._fit_ols(X, y)
        else:
            self._fit_gradient_descent(X, y)
        
        return self
    
    def _fit_ols(self, X, y):
        """Fit using Normal Equation (OLS)"""
        # Add bias column to X
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        
        # Normal equation: w = (X^T * X)^(-1) * X^T * y
        theta = np.linalg.pinv(X_b.T @ X_b) @ X_b.T @ y
        
        self.bias = theta[0]
        self.weights = theta[1:]
        
    def _fit_gradient_descent(self, X, y):
        """Fit using Gradient Descent"""
        n_samples, n_features = X.shape
        
        # Initialize parameters
        self.weights = np.zeros(n_features)
        self.bias = 0
        self.cost_history = []
        
        for i in range(self.n_iterations):
            # Predictions
            y_pred = X @ self.weights + self.bias
            
            # Compute cost
            cost = (1 / (2 * n_samples)) * np.sum((y - y_pred) ** 2)
            self.cost_history.append(cost)
            
            # Compute gradients
            dw = -(1 / n_samples) * X.T @ (y - y_pred)
            db = -(1 / n_samples) * np.sum(y - y_pred)
            
            # Update parameters
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db
    
    def predict(self, X):
        """
        Make predictions.
        
        Parameters:
        -----------
        X : array-like of shape (n_samples, n_features)
            Samples to predict
        
        Returns:
        --------
        y_pred : array of shape (n_samples,)
            Predicted values
        """
        X = np.array(X)
        if X.ndim == 1:
            X = X.reshape(-1, 1)
        return X @ self.weights + self.bias
    
    def score(self, X, y):
        """
        Calculate R² score.
        """
        y_pred = self.predict(X)
        ss_res = np.sum((y - y_pred) ** 2)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        return 1 - (ss_res / ss_tot)


# Test the implementation
print("Testing Custom Linear Regression Implementation")
print("="*60)

# Prepare data
X_train_simple = X_simple.reshape(-1, 1)

# OLS method
model_ols = LinearRegressionScratch(method='ols')
model_ols.fit(X_train_simple, y_simple)
print(f"\nOLS Method:")
print(f"  Weights: {model_ols.weights}")
print(f"  Bias: {model_ols.bias:.4f}")
print(f"  R² Score: {model_ols.score(X_train_simple, y_simple):.4f}")

# Gradient Descent method
model_gd = LinearRegressionScratch(method='gd', learning_rate=0.01, n_iterations=1000)
model_gd.fit(X_train_simple, y_simple)
print(f"\nGradient Descent Method:")
print(f"  Weights: {model_gd.weights}")
print(f"  Bias: {model_gd.bias:.4f}")
print(f"  R² Score: {model_gd.score(X_train_simple, y_simple):.4f}")

---
## 5. Scikit-learn Implementation

Now let's use scikit-learn's LinearRegression for comparison.

In [ ]:
# Scikit-learn Linear Regression
from sklearn.linear_model import LinearRegression

# Create and fit model
sklearn_model = LinearRegression()
sklearn_model.fit(X_train_simple, y_simple)

print("Scikit-learn Linear Regression")
print("="*60)
print(f"Coefficients: {sklearn_model.coef_}")
print(f"Intercept: {sklearn_model.intercept_:.4f}")
print(f"R² Score: {sklearn_model.score(X_train_simple, y_simple):.4f}")

In [ ]:
# Compare all three implementations
print("\nComparison of All Implementations")
print("="*60)
print(f"\n{'Method':<25} {'Slope':<12} {'Intercept':<12} {'R² Score':<12}")
print("-"*60)
print(f"{'Manual OLS':<25} {m_ols:<12.4f} {b_ols:<12.4f} {model_ols.score(X_train_simple, y_simple):<12.4f}")
print(f"{'Custom Class (OLS)':<25} {model_ols.weights[0]:<12.4f} {model_ols.bias:<12.4f} {model_ols.score(X_train_simple, y_simple):<12.4f}")
print(f"{'Custom Class (GD)':<25} {model_gd.weights[0]:<12.4f} {model_gd.bias:<12.4f} {model_gd.score(X_train_simple, y_simple):<12.4f}")
print(f"{'Scikit-learn':<25} {sklearn_model.coef_[0]:<12.4f} {sklearn_model.intercept_:<12.4f} {sklearn_model.score(X_train_simple, y_simple):<12.4f}")
print(f"{'True Values':<25} {2.0:<12.4f} {1.0:<12.4f} {'N/A':<12}")

In [ ]:
# Visualize predictions from all methods
plt.figure(figsize=(12, 6))

plt.scatter(X_simple, y_simple, color='blue', alpha=0.5, label='Data')
plt.plot(X_simple, model_ols.predict(X_train_simple), 'r-', linewidth=2, label='Custom OLS')
plt.plot(X_simple, model_gd.predict(X_train_simple), 'g--', linewidth=2, label='Custom GD')
plt.plot(X_simple, sklearn_model.predict(X_train_simple), 'm:', linewidth=3, label='Scikit-learn')

plt.xlabel('X', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('Comparison of Linear Regression Implementations', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---
## 6. Model Evaluation Metrics

### Key Metrics for Regression

1. **Mean Squared Error (MSE)**
   $$MSE = \frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y}_i)^2$$

2. **Root Mean Squared Error (RMSE)**
   $$RMSE = \sqrt{MSE}$$

3. **Mean Absolute Error (MAE)**
   $$MAE = \frac{1}{n}\sum_{i=1}^{n}|y_i - \hat{y}_i|$$

4. **R² Score (Coefficient of Determination)**
   $$R^2 = 1 - \frac{SS_{res}}{SS_{tot}} = 1 - \frac{\sum(y_i - \hat{y}_i)^2}{\sum(y_i - \bar{y})^2}$$

In [ ]:
# Implement evaluation metrics from scratch
def mean_squared_error_manual(y_true, y_pred):
    """Calculate Mean Squared Error"""
    return np.mean((y_true - y_pred) ** 2)

def root_mean_squared_error_manual(y_true, y_pred):
    """Calculate Root Mean Squared Error"""
    return np.sqrt(mean_squared_error_manual(y_true, y_pred))

def mean_absolute_error_manual(y_true, y_pred):
    """Calculate Mean Absolute Error"""
    return np.mean(np.abs(y_true - y_pred))

def r2_score_manual(y_true, y_pred):
    """Calculate R² Score"""
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / ss_tot)

# Calculate metrics
y_pred = sklearn_model.predict(X_train_simple)

print("Model Evaluation Metrics")
print("="*60)
print(f"\n{'Metric':<30} {'Manual':<15} {'Sklearn':<15}")
print("-"*60)
print(f"{'MSE':<30} {mean_squared_error_manual(y_simple, y_pred):<15.4f} {mean_squared_error(y_simple, y_pred):<15.4f}")
print(f"{'RMSE':<30} {root_mean_squared_error_manual(y_simple, y_pred):<15.4f} {np.sqrt(mean_squared_error(y_simple, y_pred)):<15.4f}")
print(f"{'MAE':<30} {mean_absolute_error_manual(y_simple, y_pred):<15.4f} {mean_absolute_error(y_simple, y_pred):<15.4f}")
print(f"{'R² Score':<30} {r2_score_manual(y_simple, y_pred):<15.4f} {r2_score(y_simple, y_pred):<15.4f}")

In [ ]:
# Understanding R² Score
print("\nUnderstanding R² Score")
print("="*60)
print("""
R² Score Interpretation:
- R² = 1.0: Perfect predictions (all variance explained)
- R² = 0.0: Model predicts mean value only (no variance explained)
- R² < 0.0: Model is worse than predicting mean (possible overfitting)

General Guidelines:
- R² > 0.9: Excellent fit
- 0.7 < R² < 0.9: Good fit
- 0.5 < R² < 0.7: Moderate fit
- R² < 0.5: Weak fit
""")

# Visualize R² concept
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# High R²
X_demo = np.linspace(0, 10, 50)
y_high_r2 = 2 * X_demo + 1 + np.random.normal(0, 0.5, 50)
model_temp = LinearRegression().fit(X_demo.reshape(-1, 1), y_high_r2)
axes[0].scatter(X_demo, y_high_r2, alpha=0.6)
axes[0].plot(X_demo, model_temp.predict(X_demo.reshape(-1, 1)), 'r-', linewidth=2)
axes[0].set_title(f'High R² = {model_temp.score(X_demo.reshape(-1,1), y_high_r2):.3f}', fontweight='bold')

# Medium R²
y_med_r2 = 2 * X_demo + 1 + np.random.normal(0, 3, 50)
model_temp = LinearRegression().fit(X_demo.reshape(-1, 1), y_med_r2)
axes[1].scatter(X_demo, y_med_r2, alpha=0.6)
axes[1].plot(X_demo, model_temp.predict(X_demo.reshape(-1, 1)), 'r-', linewidth=2)
axes[1].set_title(f'Medium R² = {model_temp.score(X_demo.reshape(-1,1), y_med_r2):.3f}', fontweight='bold')

# Low R²
y_low_r2 = 2 * X_demo + 1 + np.random.normal(0, 8, 50)
model_temp = LinearRegression().fit(X_demo.reshape(-1, 1), y_low_r2)
axes[2].scatter(X_demo, y_low_r2, alpha=0.6)
axes[2].plot(X_demo, model_temp.predict(X_demo.reshape(-1, 1)), 'r-', linewidth=2)
axes[2].set_title(f'Low R² = {model_temp.score(X_demo.reshape(-1,1), y_low_r2):.3f}', fontweight='bold')

for ax in axes:
    ax.set_xlabel('X')
    ax.set_ylabel('y')
    ax.grid(True, alpha=0.3)

plt.suptitle('Understanding R² Score', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---
## 7. House Price Prediction Assignment

### Task: Predict house prices using linear regression

We'll create a realistic house price dataset and build prediction models.

In [ ]:
# Create House Price Dataset
np.random.seed(42)
n_houses = 1000

# Generate features
house_data = pd.DataFrame({
    'sqft': np.random.normal(1800, 500, n_houses).clip(500, 5000),
    'bedrooms': np.random.choice([1, 2, 3, 4, 5, 6], n_houses, p=[0.05, 0.15, 0.35, 0.30, 0.10, 0.05]),
    'bathrooms': np.random.choice([1, 1.5, 2, 2.5, 3, 3.5, 4], n_houses, p=[0.10, 0.15, 0.25, 0.20, 0.15, 0.10, 0.05]),
    'age': np.random.randint(0, 100, n_houses),
    'lot_size': np.random.normal(8000, 3000, n_houses).clip(1000, 30000),
    'garage_cars': np.random.choice([0, 1, 2, 3], n_houses, p=[0.10, 0.30, 0.45, 0.15]),
    'has_pool': np.random.choice([0, 1], n_houses, p=[0.75, 0.25]),
    'neighborhood_quality': np.random.choice([1, 2, 3, 4, 5], n_houses, p=[0.05, 0.15, 0.40, 0.30, 0.10])
})

# Generate price based on features (with realistic coefficients)
house_data['price'] = (
    50000 +                                        # base price
    150 * house_data['sqft'] +                     # $150 per sqft
    15000 * house_data['bedrooms'] +               # $15k per bedroom
    20000 * house_data['bathrooms'] +              # $20k per bathroom
    -1000 * house_data['age'] +                    # -$1k per year of age
    5 * house_data['lot_size'] +                   # $5 per sqft of lot
    25000 * house_data['garage_cars'] +            # $25k per garage space
    40000 * house_data['has_pool'] +               # $40k for pool
    50000 * house_data['neighborhood_quality'] +   # $50k per quality level
    np.random.normal(0, 30000, n_houses)           # noise
).clip(50000, 2000000)

print("House Price Dataset Created")
print("="*60)
print(f"Shape: {house_data.shape}")
print("\nFirst 10 rows:")
house_data.head(10)

In [ ]:
# EDA for House Data
print("Dataset Statistics")
print("="*60)
print(house_data.describe().round(2))

In [ ]:
# Correlation analysis
plt.figure(figsize=(10, 8))
correlation = house_data.corr()
sns.heatmap(correlation, annot=True, cmap='RdYlBu_r', center=0, fmt='.2f')
plt.title('Feature Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nCorrelations with Price:")
price_corr = correlation['price'].drop('price').sort_values(ascending=False)
for feat, corr in price_corr.items():
    print(f"  {feat}: {corr:.3f}")

In [ ]:
# Scatter plots of top features vs price
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

features = ['sqft', 'bedrooms', 'bathrooms', 'age', 'lot_size', 'garage_cars', 'has_pool', 'neighborhood_quality']

for i, feat in enumerate(features):
    ax = axes[i // 4, i % 4]
    ax.scatter(house_data[feat], house_data['price']/1000, alpha=0.3)
    ax.set_xlabel(feat)
    ax.set_ylabel('Price ($K)')
    ax.set_title(f'Price vs {feat}', fontsize=10, fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.suptitle('Feature Relationships with House Price', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Prepare data for modeling
X_house = house_data.drop('price', axis=1)
y_house = house_data['price']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_house, y_house, test_size=0.2, random_state=42)

print("Data Split")
print("="*60)
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

In [ ]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Features scaled using StandardScaler")

### Model 1: Manual Implementation

In [ ]:
# Manual implementation with gradient descent
manual_model = LinearRegressionScratch(method='gd', learning_rate=0.1, n_iterations=2000)
manual_model.fit(X_train_scaled, y_train.values)

# Predictions
y_pred_manual_train = manual_model.predict(X_train_scaled)
y_pred_manual_test = manual_model.predict(X_test_scaled)

print("Manual Implementation (Gradient Descent)")
print("="*60)
print(f"\nTraining Metrics:")
print(f"  RMSE: ${np.sqrt(mean_squared_error(y_train, y_pred_manual_train)):,.2f}")
print(f"  MAE: ${mean_absolute_error(y_train, y_pred_manual_train):,.2f}")
print(f"  R²: {r2_score(y_train, y_pred_manual_train):.4f}")

print(f"\nTest Metrics:")
print(f"  RMSE: ${np.sqrt(mean_squared_error(y_test, y_pred_manual_test)):,.2f}")
print(f"  MAE: ${mean_absolute_error(y_test, y_pred_manual_test):,.2f}")
print(f"  R²: {r2_score(y_test, y_pred_manual_test):.4f}")

In [ ]:
# Plot cost convergence
plt.figure(figsize=(10, 5))
plt.plot(manual_model.cost_history)
plt.xlabel('Iteration')
plt.ylabel('Cost (MSE/2)')
plt.title('Gradient Descent Convergence - House Price Model', fontsize=12, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.show()

### Model 2: Scikit-learn Implementation

In [ ]:
# Scikit-learn implementation
sklearn_house_model = LinearRegression()
sklearn_house_model.fit(X_train_scaled, y_train)

# Predictions
y_pred_sklearn_train = sklearn_house_model.predict(X_train_scaled)
y_pred_sklearn_test = sklearn_house_model.predict(X_test_scaled)

print("Scikit-learn Implementation")
print("="*60)
print(f"\nTraining Metrics:")
print(f"  RMSE: ${np.sqrt(mean_squared_error(y_train, y_pred_sklearn_train)):,.2f}")
print(f"  MAE: ${mean_absolute_error(y_train, y_pred_sklearn_train):,.2f}")
print(f"  R²: {r2_score(y_train, y_pred_sklearn_train):.4f}")

print(f"\nTest Metrics:")
print(f"  RMSE: ${np.sqrt(mean_squared_error(y_test, y_pred_sklearn_test)):,.2f}")
print(f"  MAE: ${mean_absolute_error(y_test, y_pred_sklearn_test):,.2f}")
print(f"  R²: {r2_score(y_test, y_pred_sklearn_test):.4f}")

In [ ]:
# Feature importance (coefficients)
feature_importance = pd.DataFrame({
    'Feature': X_house.columns,
    'Coefficient_Manual': manual_model.weights,
    'Coefficient_Sklearn': sklearn_house_model.coef_
}).sort_values('Coefficient_Sklearn', key=abs, ascending=False)

print("\nFeature Coefficients (Scaled Data):")
print(feature_importance.to_string(index=False))

In [ ]:
# Visualize feature importance
fig, ax = plt.subplots(figsize=(10, 6))

feature_importance_sorted = feature_importance.sort_values('Coefficient_Sklearn')
colors = ['green' if x > 0 else 'red' for x in feature_importance_sorted['Coefficient_Sklearn']]

ax.barh(feature_importance_sorted['Feature'], feature_importance_sorted['Coefficient_Sklearn'], color=colors)
ax.axvline(x=0, color='black', linewidth=0.5)
ax.set_xlabel('Coefficient Value (Effect on Price)')
ax.set_title('Feature Importance - House Price Prediction', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

---
## 8. Comparison and Analysis

In [ ]:
# Final comparison
print("FINAL MODEL COMPARISON")
print("="*80)

results = pd.DataFrame({
    'Model': ['Manual (GD)', 'Scikit-learn'],
    'Train RMSE': [
        np.sqrt(mean_squared_error(y_train, y_pred_manual_train)),
        np.sqrt(mean_squared_error(y_train, y_pred_sklearn_train))
    ],
    'Test RMSE': [
        np.sqrt(mean_squared_error(y_test, y_pred_manual_test)),
        np.sqrt(mean_squared_error(y_test, y_pred_sklearn_test))
    ],
    'Train R²': [
        r2_score(y_train, y_pred_manual_train),
        r2_score(y_train, y_pred_sklearn_train)
    ],
    'Test R²': [
        r2_score(y_test, y_pred_manual_test),
        r2_score(y_test, y_pred_sklearn_test)
    ]
})

results['Train RMSE'] = results['Train RMSE'].apply(lambda x: f"${x:,.0f}")
results['Test RMSE'] = results['Test RMSE'].apply(lambda x: f"${x:,.0f}")
results['Train R²'] = results['Train R²'].apply(lambda x: f"{x:.4f}")
results['Test R²'] = results['Test R²'].apply(lambda x: f"{x:.4f}")

print(results.to_string(index=False))

In [ ]:
# Prediction vs Actual plots
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Manual model
axes[0].scatter(y_test, y_pred_manual_test, alpha=0.5)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Price ($)')
axes[0].set_ylabel('Predicted Price ($)')
axes[0].set_title(f'Manual Model: Predicted vs Actual\nR² = {r2_score(y_test, y_pred_manual_test):.4f}', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Sklearn model
axes[1].scatter(y_test, y_pred_sklearn_test, alpha=0.5)
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=2, label='Perfect Prediction')
axes[1].set_xlabel('Actual Price ($)')
axes[1].set_ylabel('Predicted Price ($)')
axes[1].set_title(f'Sklearn Model: Predicted vs Actual\nR² = {r2_score(y_test, y_pred_sklearn_test):.4f}', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Residual analysis
residuals_sklearn = y_test - y_pred_sklearn_test

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Residual distribution
axes[0].hist(residuals_sklearn, bins=30, edgecolor='black', alpha=0.7)
axes[0].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[0].set_xlabel('Residual ($)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Residual Distribution', fontsize=12, fontweight='bold')

# Residuals vs Predicted
axes[1].scatter(y_pred_sklearn_test, residuals_sklearn, alpha=0.5)
axes[1].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[1].set_xlabel('Predicted Price ($)')
axes[1].set_ylabel('Residual ($)')
axes[1].set_title('Residuals vs Predicted', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Q-Q plot
stats.probplot(residuals_sklearn, dist="norm", plot=axes[2])
axes[2].set_title('Q-Q Plot (Normality Check)', fontsize=12, fontweight='bold')

plt.suptitle('Residual Analysis', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Cross-validation
print("\nCross-Validation Results (5-Fold)")
print("="*60)

cv_scores = cross_val_score(sklearn_house_model, X_train_scaled, y_train, cv=5, scoring='r2')
cv_rmse = cross_val_score(sklearn_house_model, X_train_scaled, y_train, cv=5, 
                          scoring='neg_root_mean_squared_error')

print(f"\nR² Scores: {cv_scores.round(4)}")
print(f"Mean R²: {cv_scores.mean():.4f} (+/- {cv_scores.std()*2:.4f})")
print(f"\nRMSE: ${-cv_rmse.mean():,.2f} (+/- ${cv_rmse.std()*2:,.2f})")

---
## Summary

### Key Takeaways

1. **Linear Regression Basics**
   - Simple equation: y = mx + b
   - Multiple regression: y = w₀ + w₁x₁ + w₂x₂ + ...
   - Assumes linear relationship between features and target

2. **Cost Function and Optimization**
   - MSE measures prediction error
   - Gradient descent iteratively minimizes cost
   - Learning rate controls convergence speed
   - OLS provides analytical solution (faster but less flexible)

3. **Evaluation Metrics**
   - MSE/RMSE: Average squared/root error
   - MAE: Average absolute error
   - R²: Proportion of variance explained (0 to 1)

4. **Best Practices**
   - Scale features before gradient descent
   - Use train-test split for evaluation
   - Check residual plots for model assumptions
   - Use cross-validation for robust assessment

### When to Use Linear Regression

**Use When:**
- Target variable is continuous
- Relationship appears linear
- Need interpretable model
- Quick baseline model

**Avoid When:**
- Non-linear relationships
- Target is categorical
- High multicollinearity
- Complex feature interactions

In [ ]:
# Save the model for future use
import pickle

# Save sklearn model
with open('house_price_model.pkl', 'wb') as f:
    pickle.dump(sklearn_house_model, f)

# Save scaler
with open('house_price_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("Model and scaler saved!")
print("Files: house_price_model.pkl, house_price_scaler.pkl")